In [ ]:
import os
import zipfile 
import gdown
import torch
from natsort import natsorted
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

## Setup
## Number of gpus available
ngpu = 1
device = torch.device('cuda:0' if (
    torch.cuda.is_available() and ngpu > 0) else 'cpu')

## Fetch data from Google Drive 
# Root directory for the dataset
data_root = 'data/celeba'
# Path to folder with the dataset
dataset_folder = f'{data_root}/img_align_celeba'
# URL for the CelebA dataset
url = 'https://drive.google.com/uc?id=1cNIac61PSA_LqDFYFUeyaQYekYPc75NH'
#url = 'https://drive.google.com/file/d/0B7EVK8r0v71pZjFTYXZWM3FlRnM/view?usp=sharing&resourcekey=0-dYn9z10tMJOBAkviAcfdyQ'
# Path to download the dataset to
download_path = f'{data_root}/img_align_celeba.zip'

# Create required directories 
if not os.path.exists(data_root):
  os.makedirs(data_root)
  os.makedirs(dataset_folder)

# Download the dataset from google drive
gdown.download(url, download_path, quiet=False)

# Unzip the downloaded file 
with zipfile.ZipFile(download_path, 'r') as ziphandler:
  ziphandler.extractall(dataset_folder)

In [ ]:
filenames_list = []
dataset_dir = f'{data_root}/img_align_celeba/img_align_celeba/'

for filename in sorted(os.listdir(dataset_dir)):
    filenames_list.append(dataset_dir + filename)

filenames_list_train = filenames_list[:100000]
filenames_list_test = filenames_list[100001:120000]

print(len(filenames_list))

In [ ]:
filenames_list_train[0]

In [ ]:
# Super Resolution class
import numpy as np
#from keras.utils import Sequence
from tensorflow.keras.utils import Sequence

from skimage import io, color
from skimage.transform import resize
from matplotlib import pyplot as plt

img_size = 128

class Generator(Sequence):

    def __init__(self, filenames_list, batch_size, img_size):
        self.filenames_list = filenames_list
        self.batch_size = batch_size
        self.img_size = img_size

    def __len__(self):
        return int(np.ceil(len(self.filenames_list) / float(self.batch_size)))

    def __getitem__(self, idx):

        batch_x = self.filenames_list[idx * self.batch_size:(idx + 1) * self.batch_size]

        input_tensor = np.empty((self.batch_size, int(self.img_size/2), int(self.img_size/2), 3), dtype=np.float32)
        output_tensor = np.empty((self.batch_size, self.img_size, self.img_size, 3), dtype=np.float32)

        i = 0
        for filename in batch_x:

            read_image = io.imread(filename)

            image = resize(read_image, (self.img_size, self.img_size), anti_aliasing=False, mode='constant')
            image_n = resize(read_image, (int(self.img_size/2), int(self.img_size/2)), anti_aliasing=False, mode='constant')

            if image.shape == (self.img_size, self.img_size, 3) :
                # array image for output tensor
                output_tensor[i, :] = (image[:, :, :])
                # array values for input tensor
                input_tensor[i, :] = (image_n[:, :, :] ).reshape(int(self.img_size/2), int(self.img_size/2), 3)
                i += 1

                # print(image)
                # io.imshow(image_n)
                # plt.show()
                # io.imshow(image)
                # plt.show()


        return input_tensor, output_tensor


In [ ]:
import os
import tensorflow as tf
import tensorflow.keras
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Activation, Input, ZeroPadding2D, Concatenate, AveragePooling2D, UpSampling2D

from time import time
from tensorflow.python.keras.callbacks import TensorBoard
#from keras.utils import plot_model
import matplotlib.pyplot as plt

In [ ]:
!nvidia-smi

In [ ]:
# Root directory for the logs
log_root = f'{data_root}/logs'
# Path to folder with the dataset
weights_root = f'{data_root}//weights'

# Create required directories 
if not os.path.exists(log_root):
  os.makedirs(log_root)
  os.makedirs(weights_root)

In [ ]:
img_size = 64
batch_size = 64


training_batch_generator = Generator(filenames_list=filenames_list_train, batch_size=batch_size, img_size=img_size*2)
testing_batch_generator = Generator(filenames_list=filenames_list_test, batch_size=batch_size, img_size=img_size*2)
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))


In [ ]:
# model
input_layer = Input(shape=(img_size, img_size, 3))

hidden_layer_1 = Conv2D(32, (7, 7), activation='relu', padding='same')(input_layer)
hidden_layer_1 = BatchNormalization()(hidden_layer_1)
hidden_layer_1 = AveragePooling2D()(hidden_layer_1)
hidden_layer_2 = Conv2D(64, (5, 5), activation='relu',  padding='same')(hidden_layer_1)
hidden_layer_2 = BatchNormalization()(hidden_layer_2)
hidden_layer_2 = AveragePooling2D()(hidden_layer_2)
hidden_layer_2 = UpSampling2D()(hidden_layer_2)
hidden_layer_3 = Conv2D(256, (3, 3), activation='relu', padding='same')(hidden_layer_2)
hidden_layer_3 = BatchNormalization()(hidden_layer_3)
hidden_layer_3 = UpSampling2D()(hidden_layer_3)
hidden_layer_4 = Conv2D(64, (3, 3), activation='relu', padding='same')(hidden_layer_3)
hidden_layer_4 = BatchNormalization()(hidden_layer_4)
hidden_layer_4 = UpSampling2D()(hidden_layer_4)
hidden_layer_5 = Conv2D(32, (3, 3), activation='relu', padding='same')(hidden_layer_4)
hidden_layer_5 = BatchNormalization()(hidden_layer_5)

output_layer = Conv2D(3, (3, 3), activation='tanh', padding='same')(hidden_layer_5)

model = Model(inputs=input_layer, outputs=output_layer)

opt = tensorflow.keras.optimizers.RMSprop()

model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])

#plot_model(model, to_file='logs/cnn/cnn_model.png')
print(model.summary())

In [ ]:
with tf.device("/gpu:0"):
   history = model.fit(training_batch_generator, epochs=10, verbose=1,workers=8,validation_data=testing_batch_generator)

   print(history.history.keys())
   #summarize history for accuracy
   plt.plot(history.history['accuracy'])
   #plt.plot(history.history['val_accuracy'])
   plt.title('model accuracy')
   plt.ylabel('accuracy')
   plt.xlabel('epoch')
   plt.legend(['train'], loc='upper left')
   plt.savefig(log_root+"/accuracy_plot.png")
   plt.show()
   # summarize history for loss
   plt.plot(history.history['loss'])
   #plt.plot(history.history['val_loss'])
   plt.title('model loss')
   plt.ylabel('loss')
   plt.xlabel('epoch')
   plt.legend(['train'], loc='upper left')
   plt.savefig(log_root+"/loss_plot.png")
   plt.show()
  #summarize history for accuracy test and train
   plt.plot(history.history['accuracy'])
   plt.plot(history.history['val_accuracy'])
   plt.title('model accuracy')
   plt.ylabel('accuracy')
   plt.xlabel('epoch')
   plt.legend(['train', 'test'], loc='upper left')
   plt.savefig(log_root+"/accuracy_val_plot.png")
   plt.show()
   # summarize history for loss for test and train
   plt.plot(history.history['loss'])
   plt.plot(history.history['val_loss'])
   plt.title('model loss')
   plt.ylabel('loss')
   plt.xlabel('epoch')
   plt.legend(['train', 'test'], loc='upper left')
   plt.savefig(log_root+"/loss_val_plot.png")
   plt.show()

model.save(weights_root+"/AECNNs.h5")

In [ ]:
import numpy as np
import os
from keras.models import load_model
import matplotlib.pyplot as plt
from skimage import io, color , util
from skimage.transform import resize
import pandas as pd

In [ ]:
img_size = 128
dataset_dir = f'{data_root}/img_align_celeba/img_align_celeba/'

def ret_input_output_tensor(dataset_dir, n_images):
    input_tensor = np.empty((n_images, int(img_size/2), int(img_size/2), 3))
    output_tensor = np.empty((n_images, img_size, img_size, 3))

    i = 0
    print(len(os.listdir(dataset_dir)))
    for filename in reversed(sorted(os.listdir(dataset_dir))):

        if filename.endswith('.jpg'):

            read_image = io.imread(dataset_dir + filename)
            image = resize(read_image, (img_size, img_size, 3), anti_aliasing=False, mode='constant')
            image_n = resize(read_image, (int(img_size/2), int(img_size/2)), anti_aliasing=False, mode='constant')


            if image.shape == (img_size, img_size, 3):
                # array image for output tensor
                output_tensor[i, :] = (image[:, :, :])
                # array values for input tensor
                input_tensor[i, :] = (image_n[:, :, :]).reshape(int(img_size/2), int(img_size/2), 3)
                i += 1

        if i >= n_images: break

    return input_tensor, output_tensor

In [ ]:
print(os.listdir())
model = load_model("/content/data/AECNNs.h5")

# prediction

input_tensor, output_tensor = ret_input_output_tensor(dataset_dir, 10)
prediction = model.predict(input_tensor)

In [ ]:
for i in range(10):
    print("input tensor :\n", input_tensor[i].shape)
    print("prediction :\n", prediction[i].shape)
    print("output tensor :\n", output_tensor[i].shape)

    input_image = input_tensor[i]
    print(input_image.shape)
    plt.imshow(input_image)
    plt.show()
    plt.imsave('/content/data/celeba/logs/'+str(i)+'_input.png', input_image)

    output_image = prediction[i]
    print(output_image.shape)

    plt.imshow(output_image)
    plt.show()
    #plt.savefig('/content/data/celeba/logs/' + str(i) + '_prediction.png')
    #plt.imsave('/content/data/celeba/logs/' + str(i) + '_prediction.png', output_image)



    output_image = output_tensor[i]
    plt.imshow(output_image)
    plt.show()
    plt.imsave('/content/data/celeba/logs/' + str(i) + '_actual.png', output_image)

In [ ]:
!zip -r /content/data/logs.zip /content/data/celeba/logs